In [2]:
from datasets import load_dataset
from tqdm import tqdm
from collections import defaultdict

In [65]:
model.save_model("model_filename_3gram.bin")


In [138]:
imdb = load_dataset("imdb")

In [3]:
import unicodedata
import re

def normalize_for_hashing(text):
    text = text.replace('<SEP>', ' ')
    text = text.replace('<PAD>', '')
    text = unicodedata.normalize('NFKD', text)
    text = text.encode('ASCII', 'ignore').decode('utf-8')
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = text.strip()
    text = text.lower()
    return text

In [232]:
vs = imdb['test']

In [245]:
ds = imdb['train']

In [225]:
def transform(examples):
    label_prefix = '__label__'
    i2l = ['neg', 'pos']
    examples['new_label'] = [ label_prefix + i2l[l] for l in examples['label'] ]
    examples['text'] = [normalize_for_hashing(txt) for txt in examples['text']]
    return examples



In [226]:
ds=ds.map(transform,batched=True,num_proc=4)

Map (num_proc=4):   0%|          | 0/25000 [00:00<?, ? examples/s]

In [235]:
vs=vs.map(transform,batched=True,num_proc=4)

Map (num_proc=4):   0%|          | 0/25000 [00:00<?, ? examples/s]

In [236]:
with open('imbd_test.txt', 'w') as f:
    for x in tqdm(vs):
        print(x['new_label'] + ' ' + x['text'], file=f)
with open('imbd_train.txt', 'w') as f:
    for x in tqdm(ds):
        print(x['new_label'] + ' ' + x['text'], file=f)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25000/25000 [00:00<00:00, 79549.67it/s]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25000/25000 [00:00<00:00, 81265.12it/s]


In [246]:
ds[0]['text']

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, ev

In [228]:
ds[0]['new_label']

'__label__neg'

In [239]:
import fasttext

model = fasttext.train_supervised('imbd_train.txt',epoch=50)

Read 5M words
Number of words:  120629
Number of labels: 2
Progress: 100.0% words/sec/thread: 6325646 lr:  0.000000 avg.loss:  0.102371 ETA:   0h 0m 0s


In [60]:
import fasttext

model = fasttext.train_supervised('quality_train.txt',epoch=5,wordNgrams=3)

Read 108M words
Number of words:  3059267
Number of labels: 2
Progress: 100.0% words/sec/thread: 1432808 lr:  0.000000 avg.loss:  0.080174 ETA:   0h 0m 0s


In [240]:
model.predict(normalize_for_hashing(vs[0]['text']))

(('__label__neg',), array([1.00000942]))

In [241]:
vs[0]['text']

'i love scifi and am willing to put up with a lot scifi moviestv are usually underfunded underappreciated and misunderstood i tried to like this i really did but it is to good tv scifi as babylon 5 is to star trek the original silly prosthetics cheap cardboard sets stilted dialogues cg that doesnt match the background and painfully onedimensional characters cannot be overcome with a scifi setting im sure there are those of you out there who think babylon 5 is good scifi tv its not its cliched and uninspiring while us viewers might like emotion and character development scifi is a genre that does not take itself seriously cf star trek it may treat important issues yet not as a serious philosophy its really difficult to care about the characters here as they are not simply foolish just missing a spark of life their actions and reactions are wooden and predictable often painful to watch the makers of earth know its rubbish as they have to always say gene roddenberrys earth otherwise peopl

In [62]:
def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

In [ ]:
print_results(*model.test('imbd_test.txt'))

In [63]:
print_results(*model.test('quality_test.txt'))

N	40000
P@1	0.982
R@1	0.982


In [244]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [247]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)
tokenized_imdb = imdb.map(preprocess_function, batched=True)


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [254]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [248]:
import evaluate

accuracy = evaluate.load("accuracy")

In [249]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [255]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

In [140]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Lower batch size
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=4,  # Simulate a larger batch size
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to=[]
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

NameError: name 'TrainingArguments' is not defined

IterableDataset({
    features: ['text'],
    num_shards: 1555
})

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

In [3]:
xs = []
progress_bar = tqdm(total=10000, desc="Processing")
for x in the_pile:
    if x['meta']['pile_set_name'] in clean_text:
        xs.append(len(x['text']))
        #print(len(xs))
        progress_bar.update(1)
        if len(xs)>10000:
            progress_bar.close()
            break

Processing:   0%|                                                                                                                                                                                                     | 0/10000 [00:00<?, ?it/s]

NameError: name 'the_pile' is not defined

In [342]:
np.array(xs).mean()

np.float64(4296.908109189081)

In [351]:
xs = []
progress_bar = tqdm(total=10000, desc="Processing")
for x in noisy:
    xs.append(len(x['text']))
    progress_bar.update(1)
    if len(xs)>10000:
        progress_bar.close()
        break

Processing: 10001it [00:05, 1770.07it/s]                                                                                                                                                                                                        


In [352]:
np.array(xs).mean()

np.float64(4198.1552844715525)

In [ ]:
with open('quality_train.txt', 'w') as f:
    progress_bar = tqdm(total=N_DS, desc="Processing")
    for x in the_pile:
        if :
            text = normalize_for_hashing(x['text'])
            print('__label__pos'+' '+text, file=f)

    progress_bar = tqdm(total=10000, desc="Processing")
    for x in noisy:
        progress_bar.update(1)
        text = normalize_for_hashing(x['text'])
        print('__label__neg'+' '+text, file=f)
    
        

In [68]:
"\1"

'\x01'

In [70]:
import string
punctuation_pat = re.compile(f"([{re.escape(string.punctuation)}])")
whitespace_pat = re.compile(r"\s+")


def preprocess_for_fasttext(text):
    """Preprocess text for fastText classification."""
    text = text.replace('\\n', '\n')
    text = text.lower()
    text = punctuation_pat.sub(r" \1 ", text)
    text = whitespace_pat.sub(" ", text)
    text = text.strip()
    return text

In [76]:
print(preprocess_for_fasttext(x[25]['text']))

floorplan finder - campkin ' s rv centre narrow down the floorplans best suited to your needs . . . our new floorplan finder is a tool developed to help first - time rv buyers narrow down the hundreds of brands , types and models available in our inventory . the tool is based on a series of questions , and even if you don’t know some of the answers , the form will generate a list of inventory best suited to your requirements . after answering the questions click the orange submit button , and your personal list of models will load . first we need to determine if you are looking to travel with your trailer , towing it from site to site , or if you are planning to park it at a camp grounds and live or vacation at that same location ? tow park both do you know what body style you are interested in ? what is the maximum weight your current tow vehicle can handle , or if you plan on purchasing a new tow vehicle , what is the planned maximum weight you are looking to tow ? what is the maximu

In [128]:
x=list(itertools.islice(noisy,50))



'(ReadTimeoutError("HTTPSConnectionPool(host='cdn-lfs.hf.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 00b08ed2-8412-4cc0-864a-dc4b23b20f17)')' thrown while requesting GET https://huggingface.co/datasets/allenai/MADLAD-400/resolve/ecd71297d60c1eb996cd3d7c44c60ad5b55adfc6/data/en/en_noisy_0000.jsonl.gz
Retrying in 1s [Retry 1/5].


In [137]:
print(x[2]['text'].replace('\\n', '\n'))

India Malayalam Pillai Mca Grooms - Boys - India Malayalam Pillai Mca Groom Matrimony - PillaiMatrimony.com
Pillai Matrimony > Malayalam > India > Mca Matrimony
Pillai India MCA Grooms
My name is Anuraj M Pillai and I am working as a Officer in the Private sector. I have completed my MCA. I grew up in a Upper Middle Class class, Nuclear family with Moderate values.


In [115]:
x = list(itertools.islice(the_pile_cleaned,50))

In [ ]:
def clean_wikipedia(article):

In [127]:
print(x[40]['text'][:5000])

HotKnot is a near-field communication technology (which is mainly used in a capacitive touch screen) used in some smart terminal devices. This near-field communication includes two processes: proximity detection process and data transmission process. The proximity detection process of the near-field communication is: a touch screen terminal of one party sends a proximity detection sequence (for example, the proximity detection sequence includes six frequencies), and after receiving the proximity detection sequence, a touch screen terminal of the other party successively scans the multiple frequencies included in the proximity detection sequence. If signal strength at each frequency is greater than a preset signal strength threshold, it is considered that a signal source exists at the frequency. After the scan is completed, if signal sources exist at all frequencies, it is determined that the sequence is valid; otherwise, it is determined that the sequence is invalid. After it is determ

In [13]:
N_DS=3

In [7]:
N_DS=30_000

In [52]:
N_DS=1_000_000

In [114]:
clean_text = ['USPTO Backgrounds', 'Wikipedia (en)', 'Gutenberg (PG-19)']
clean_text = ['Gutenberg (PG-19)']
clean_text = ['USPTO Backgrounds']

def filter_clean_examples(examples):
    return [ x['pile_set_name'] in clean_text for x in examples['meta'] ]

the_pile = load_dataset("monology/pile-uncopyrighted", split="train", streaming=True)

the_pile_cleaned = the_pile.filter(filter_clean_examples, batched=True)

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

In [5]:
noisy = load_dataset("allenai/madlad-400", "en", split="noisy", streaming=True)
noisy

IterableDataset({
    features: ['text'],
    num_shards: 1555
})

In [ ]:
noisy

In [8]:
import itertools
clean_pile_iterator = itertools.islice(the_pile_cleaned,N_DS)
noisy_iterator = itertools.islice(noisy,N_DS)

In [9]:
labels = []
texts = []

for x in tqdm(clean_pile_iterator, total=N_DS, position=0, leave=True):
    text = normalize_for_hashing(x['text'])
    texts.append(text)
    labels.append('__label__pos')
    
for x in tqdm(noisy_iterator, total=N_DS, position=0, leave=True):
    text = normalize_for_hashing(x['text'])
    texts.append(text)
    labels.append('__label__neg')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30000/30000 [00:20<00:00, 1484.85it/s]


In [10]:
from datasets import Dataset
ds = Dataset.from_dict({'label': labels, 'text': texts})

In [11]:
ds=ds.train_test_split(test_size=0.2)

In [12]:
ds

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 48000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 12000
    })
})

In [13]:
print('test')
with open('quality_long_test.txt', 'w') as f:
    for x in tqdm(ds['test'], position=0, leave=True):
        print(x['label'] + ' ' + x['text'], file=f)

print('train')
with open('quality_long_train.txt', 'w') as f:
    for x in tqdm(ds['train'], position=0, leave=True):
        print(x['label'] + ' ' + x['text'], file=f)

test


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12000/12000 [00:00<00:00, 48609.37it/s]


train


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48000/48000 [00:01<00:00, 47838.17it/s]


In [33]:
idx = 19
print(ds['train'][idx]['label'])
print(ds['train'][idx]['text'])

__label__neg
100hr mindfulness  meditation teacher program  higher potential livingnhigher potential livingn100hr mindfulness  meditation teacher training programn100hr mindfulness  meditationnmay 20th  july 22nd 2021na mindfulness  meditation teacher training is perfect for practitioners who are looking to share their knowledge and experience with others a personal practice of mindfulness and meditation is essential for moving forward with a formal training and in this course you will learn methods and techniques that suit a wide variety of learning styles and will be given the setting to dive deeper into your own exploration and developmentnthis course is intended for practitioners who already have a solid foundation in mindfulness and meditation who are looking to take the next step into teaching we are looking for students who have experienced one or more of the followingn200hr or more yoga teacher trainingnregular mindfulness  meditation practice accompanied with self studynsessio

In [ ]:
with open('quality_train.txt', 'a') as f:
    

In [ ]:
sum(1 for x in tqdm(b,total=N_DS) if x['meta']['pile_set_name'] in clean_text)

In [345]:
it = iter(noisy)

In [350]:
sample = next(it)

In [349]:
list(sample.keys())

['text']

In [330]:
field = 'text'
field = 'EN'

In [331]:
len(sample[field])

620596

In [333]:
print(sample[field][:3700].replace('\\n', '\n'))

CHAPTER I. START IN LIFE


I was born in the year 1632, in the city of York, of a good family,
though not of that country, my father being a foreigner of Bremen, who
settled first at Hull. He got a good estate by merchandise, and leaving
off his trade, lived afterwards at York, from whence he had married my
mother, whose relations were named Robinson, a very good family in that
country, and from whom I was called Robinson Kreutznaer; but, by the
usual corruption of words in England, we are now called nay we call
ourselves and write our name Crusoe; and so my companions always called
me.

I had two elder brothers, one of whom was lieutenant-colonel to an
English regiment of foot in Flanders, formerly commanded by the famous
Colonel Lockhart, and was killed at the battle near Dunkirk against the
Spaniards. What became of my second brother I never knew, any more than
my father or mother knew what became of me.

Being the third son of the family and not bred to any trade, my head
began to 

In [192]:
print(sample['EN'][:1000])

CHAPTER I.

THE IMPOSSIBLE--INEVITABLE.


In the garden the question was settled without serious difference of
opinion. If Sir Robert Perry really could not go on--and Lady Eynesford
was by no means prepared to concede even that--then Mr. Puttock,
_bourgeois_ as he was, or Mr. Coxon, conceited and priggish though he
might be, must come in. At any rate, the one indisputable fact was the
impossibility of Mr. Medland: this was, to Lady Eynesford's mind,
axiomatic, and, in the safe privacy of her family circle (for Miss
Scaife counted as one of the family, and Captain Heseltine and Mr.
Flemyng did not count at all), she went so far as to declare that, let
the Governor do as he would (in the inconceivable case of his being so
foolish as to do anything of the kind), she at least would not receive
Mr. Medland. Having launched this hypothetical thunderbolt, she asked
Alicia Derosne to give her another cup of tea. Alicia poured out the
tea, handed it to her sister-in-law, and asked,

"But, Mary

In [47]:
unique_pile_set_names = set()

In [275]:
LIMIT = 10_000
sns = defaultdict(list)
for i,sample in tqdm(enumerate(the_pile), total=LIMIT):
    if len(sns[sample['meta']['pile_set_name']])<10:
        sns[sample['meta']['pile_set_name']].append(sample['text'])
    if i>LIMIT:
        break
    

10001it [00:05, 1816.47it/s]                                                                                                                                                                                                                    


In [276]:
k=list(sns.keys())
list(enumerate(k))

[(0, 'Pile-CC'),
 (1, 'Github'),
 (2, 'StackExchange'),
 (3, 'Wikipedia (en)'),
 (4, 'PubMed Abstracts'),
 (5, 'USPTO Backgrounds'),
 (6, 'FreeLaw'),
 (7, 'PubMed Central'),
 (8, 'Enron Emails'),
 (9, 'HackerNews'),
 (10, 'NIH ExPorter'),
 (11, 'ArXiv'),
 (12, 'DM Mathematics'),
 (13, 'Ubuntu IRC'),
 (14, 'EuroParl'),
 (15, 'PhilPapers'),
 (16, 'Gutenberg (PG-19)')]

In [283]:
print(sns['Pile-CC'][6][:1000])

jOOQ on The ORM Foundation?

I am the developer of jOOQ, a Java database abstraction framework. I was wondering whether jOOQ might be an interesting tool for discussion on your website, even if it is not exactly an ORM in the classic meaning (as in mapping objects to the relational world > ORM). Instead, jOOQ uses a reverse engineering paradigm (as in mapping relational entities to objects > "ROM").

Re: jOOQ on The ORM Foundation?

Object Role Modeling (the original ORM) is not the same thing as Object/Relational Mapping.

Object/Relational Mapping is still kind-of relevant and interesting to us, since Object Role Modeling is used to design databases (which then will require programmatic access). But there are probably better places to discuss it :]

Your query DSL looks rather like some of the DSLs available for Ruby, such as through the Sequel gem, or Arel. Interesting to see how well that can work with a statically-types language like Java. Maybe you or I should make a generator fo

In [ ]:
print(noisy)


In [ ]:
for k,v in .items():
    print(k)
    print(v[:1000])
    print('====================')

In [33]:
unique_pile_set_names

{'ArXiv',
 'DM Mathematics',
 'Enron Emails',
 'EuroParl',
 'FreeLaw',
 'Github',
 'Gutenberg (PG-19)',
 'HackerNews',
 'NIH ExPorter',
 'PhilPapers',
 'Pile-CC',
 'PubMed Abstracts',
 'PubMed Central',
 'StackExchange',
 'USPTO Backgrounds',
 'Ubuntu IRC',
 'Wikipedia (en)'}

In [15]:
'pile_set_name' in sample

False

In [60]:
en = load_dataset("allenai/c4", "en.noclean", streaming=True)


Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/7168 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/64 [00:00<?, ?it/s]

In [ ]:
import random
count = 0
for x in en['train']:
    if random.uniform(0,1) < 0.01:
        print(x['text'])
        print('=========================')
        count+=1
    if count>30:
        break


In [ ]:
print(sample['text'])